In [33]:
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import MmCorpus
from gensim.corpora import Dictionary
from collections import Counter, defaultdict
from numpy.random import seed
from metadata.metadata import ACL_metadata
from _topic_modeling.lda_loader import Loader
import _pickle as pkl
from metadata import Gender
import matplotlib.pyplot as plt
import numpy as np
import logging
import gensim 
seed(1)
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [2]:
loader = Loader()

INFO:gensim.corpora.indexedcorpus:loaded corpus index from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow.mm.index
INFO:gensim.matutils:initializing corpus reader from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow.mm
INFO:gensim.matutils:accepted corpus with 23595 documents, 60434 features, 16454656 non-zero entries
INFO:gensim.utils:loading LdaMallet object from /home/ramona/Desktop/Honours-LDA/aan/../malltepy
INFO:gensim.utils:loaded /home/ramona/Desktop/Honours-LDA/aan/../malltepy


In [3]:
lda = gensim.models.ldamodel.LdaModel.load('../models/model500.pkl')

INFO:gensim.utils:loading LdaModel object from ../models/model500.pkl
INFO:gensim.utils:loading expElogbeta from ../models/model500.pkl.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute id2word to None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loaded ../models/model500.pkl
INFO:gensim.utils:loading LdaModel object from ../models/model500.pkl.state
INFO:gensim.utils:loaded ../models/model500.pkl.state


In [4]:
lda.id2word = loader.id2word

In [5]:
lda.get_document_topics(loader.corpus[1])

[(0, 0.025419349521797297),
 (6, 0.025850081552067954),
 (29, 0.03496036117932748),
 (39, 0.01566633148413228),
 (48, 0.01783185430878628),
 (54, 0.15960324726529973),
 (59, 0.0560926414138563),
 (61, 0.01634006535868862),
 (67, 0.10597677929556155),
 (70, 0.31895346351565473),
 (83, 0.1131325160571294),
 (90, 0.059135231314702316)]

In [6]:
len(loader.id2word.keys())

60434

In [30]:
corpus = loader.corpus

In [24]:
gensimcm = CoherenceModel(model=lda, corpus=loader.corpus, dictionary=loader.dic, coherence='u_mass')

In [8]:
acl = ACL_metadata()
df = acl.meta_df

In [9]:
for i, p in lda.get_topic_terms(38):
    print(loader.id2word[i])

tile
tag
derivation
tree
grammar
ill
adjoin
elementary
elementary_tree
auxiliary


In [50]:
lda.show_topic(21,50)

[('dik', 0.08976261938500821),
 ('related', 0.0007147329916274371),
 ('help', 0.0006987085254538772),
 ('mark', 0.0006955483797717728),
 ('extend', 0.0006634935823047455),
 ('directly', 0.0006452456565045284),
 ('additional', 0.0006156192029361547),
 ('thank', 0.0005738868998248706),
 ('final', 0.0005368903341675367),
 ('focus', 0.0005129074460133526),
 ('multiple', 0.0005096771108182055),
 ('tain', 0.0005075608337405172),
 ('following', 0.0005003175884311681),
 ('pose', 0.0004880650916312303),
 ('reason', 0.0004843616879787706),
 ('version', 0.00047658690384412675),
 ('remain', 0.0004765717282190762),
 ('employ', 0.0004760039906302475),
 ('one', 0.00047411062109589935),
 ('rest', 0.00046813829054818804),
 ('rely', 0.00046676531257057826),
 ('tend', 0.0004636954445911725),
 ('respectively', 0.00046303477417796997),
 ('ne', 0.00046229374780276347),
 ('usually', 0.0004499605943230428),
 ('start', 0.00044969911449722785),
 ('furthermore', 0.00044721037043627147),
 ('treat', 0.000441312047

In [32]:
#doc_topics = list(map(lambda x: lda.get_document_topics(x), corpus))

In [41]:
# with open("../doc_topics_gensim.pkl","wb") as f:
#     pkl.dump(doc_topics, f)
with open("../doc_topics_gensim.pkl","rb") as f:
    doc_topics = pkl.load(f)

In [37]:
females_topics = []
females_topics_proportions = defaultdict(int) # P(topic|gender)'
males_topics_proportions = defaultdict(int)
fem_norm = 0 # normalizer
male_norm = 0
males_topics = []
for i, file in enumerate(acl.train_files):
    try:
        topics = doc_topics[i]
        gender = df.loc[acl.get_id(file)]["genders"][0] #first author
        if gender == Gender.female:
            for topic, p in topics:
                females_topics_proportions[topic] += p
            fem_norm +=1
        elif gender == Gender.male:
            for topic, p in topics:
                males_topics_proportions[topic] += p
            male_norm +=1
    except KeyError:
        pass

In [45]:
doc_topics[21]

[(0, 0.025664048800330137),
 (3, 0.1472034337219437),
 (8, 0.015802955455413117),
 (15, 0.0562709383421671),
 (25, 0.014178564005336527),
 (27, 0.011599366266778259),
 (39, 0.3283978219092855),
 (51, 0.018097345193582897),
 (64, 0.029926796385942953),
 (95, 0.298751496969276)]

In [53]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (f * (1-f)) / (m * (1-m))
    if odds > 1.5:
        print("Females are {0:.2f} more likeky than men to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)


Females are 1.54 more likeky than men to write about topic 4
Most common words for this topic are:
emotion
affect
emotional
social
people
attitude
gender
quote
male
character
------------------------------
------------------------------
Females are 1.57 more likeky than men to write about topic 19
Most common words for this topic are:
verb
subject
object
argument
verbs
construction
frame
complement
obj
verbal
------------------------------
------------------------------
Females are 2.66 more likeky than men to write about topic 21
Most common words for this topic are:
dik
related
help
mark
extend
directly
additional
thank
final
focus
------------------------------
------------------------------
Females are 1.65 more likeky than men to write about topic 23
Most common words for this topic are:
cue
scope
negation
hedge
modality
uncertainty
negate
detection
statement
certainty
------------------------------
------------------------------
Females are 4.87 more likeky than men to write abou

In [54]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (m * (1-m)) / (f * (1-f))
    if odds > 1.5:
        print("Males are {0:.2f} more likeky than females to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)


Males are 2.42 more likeky than females to write about topic 20
Most common words for this topic are:
grammar
parse
parsing
parser
free
string
context_free
symbol
chart
item
------------------------------
------------------------------
Males are 1.86 more likeky than females to write about topic 24
Most common words for this topic are:
japanese
korean
expression
japan
proc
accuracy
dictionary
tokyo
thai
kanji
------------------------------
------------------------------
Males are 1.58 more likeky than females to write about topic 29
Most common words for this topic are:
sequence
position
length
right
left
fig
block
element
line
index
------------------------------
------------------------------
Males are 1.58 more likeky than females to write about topic 37
Most common words for this topic are:
ccg
logical
logical_form
logic
parsing
predicate
steedman
parser
derivation
mooney
------------------------------
------------------------------
Males are 1.61 more likeky than females to write 